In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.feature import graycomatrix, graycoprops
from scipy.fft import fft2
from scipy.ndimage import sobel
import os

folder = 'visioncomputacional\\imagenesBrodatzCompuestas\\imagenesBrodatzCompuestas'
valid_exts = ('.png', '.jpg', '.jpeg', '.bmp')  # extensiones válidas
image_paths = [os.path.join(folder, fname) for fname in os.listdir(folder) if fname.lower().endswith(valid_exts)]

def load_images(image_paths):
    images = []
    names = []
    for path in image_paths:
        try:
            image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"Una de las imágenes es nula: {path}")
            else:
                name = os.path.basename(path)
                images.append(image) 
                names.append(name)  # Aquí también añadimos el nombre
        except Exception as e:
            print(f"Error al cargar la imagen '{path}': {e}") 
    return images, names

#Equalización de las imágenes
def equalize_image(img):
    #si estamos en escala de grises
    if len(img.shape)==2:
        return cv2.equalizeHist(img) 
    #si la imagen tiene tres canales
    elif len(img.shape)==3:
        eq_channels = []
        channels = cv2.split(img) #separamos los canales 
        for c in channels:
            ecualized = cv2.equalizeHist(c)
            eq_channels.append(ecualized) 
        return cv2.merge(eq_channels)
    else:
        print("Ecualizacion fallida, formato no esperado")

#Extracción de Caracteristicas GLCM
def feathure_extraction(channel):
    d = [1, 2, 4]
    ang = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # en las cuatro direcciones 
    glcm = graycomatrix(channel, distances=d, angles=ang,
                         levels=256, symmetric=True, normed=True)
    features = []
    for prop in ['contrast', 'energy', 'homogeneity', 'correlation']: 
        stat = graycoprops(glcm, prop)  # Calculamos la propiedad para cada ángulo
        features.extend([np.mean(stat), np.std(stat)])
    return features


# Frecuencia, escala y orientación
def extract_additional_features(image):
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    f = np.abs(fft2(gray))
    freq_mean = np.mean(f)
    freq_std = np.std(f)
    grad_x = sobel(gray, axis=0)
    grad_y = sobel(gray, axis=1)
    orientation = np.arctan2(grad_y, grad_x)
    orientation_mean = np.mean(orientation)
    orientation_std = np.std(orientation)
    return [freq_mean, freq_std, orientation_mean, orientation_std]

# Vector completo
def build_feature_vector(image):
    image = equalize_image(image)
    features = []
    if len(image.shape) == 3:
        for i in range(3):
            features += feathure_extraction(image[:, :, i])
    else:
        features += feathure_extraction(image)
    features += extract_additional_features(image)
    return features


images,names = load_images(image_paths)

all_feathures = []

for img in images:
    res = build_feature_vector(img)
    all_feathures.append(res)

print(f"Se generaron {len(all_feathures)} vectores de características.")

